In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from pyfume.Clustering import Clusterer
from pyfume.EstimateAntecendentSet import AntecedentEstimator
from pyfume.EstimateConsequentParameters import ConsequentEstimator
from pyfume.SimpfulModelBuilder import SugenoFISBuilder
from pyfume.Tester import SugenoFISTester
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, cohen_kappa_score
from numpy import clip, column_stack, argmax

In [26]:
column_names = ["Clump Thickness", "Uniformity of Cell Size", "Uniformity of Cell Shape", "Marginal Adhesion", "Single Epithelial Cell Size", "Bland Chromatin", "Normal Nucleoli", "Mitoses", "Something", "Cancer"]
column_names_x = ["Clump Thickness", "Uniformity of Cell Size", "Uniformity of Cell Shape", "Marginal Adhesion", "Single Epithelial Cell Size", "Bland Chromatin", "Normal Nucleoli", "Mitoses", "Something"]

data = pd.read_csv(os.path.abspath('wbco.csv'),names=column_names)
data.head()
data.replace("?", "1", inplace=True)

#Create the x and y variables
X = data[column_names_x].values
y = data["Cancer"].values.reshape(-1, 1)
y = np.squeeze(y)

In [23]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.array
y_test = y_test.array

AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [22]:
# Cluster the input-output space
cl = Clusterer(x_train=X_train, y_train=y_train, nr_clus=10)
clust_centers, part_matrix, _ = cl.cluster(method='fcm')

TypeError: can't multiply sequence by non-int of type 'float'